Импорт библиотек

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import telebot
from telebot import types

Подключение к таблице для создания датафрейма

In [2]:
df_shedule = pd.read_excel('SheduleFESU.xlsx', header=0, usecols='A:I')
df_shedule.loc[df_shedule["Profile"] == ' Социология', "Profile"] = "Социология" #df = df.replace(to_replace=r'.*', value=new_value, regex=True)
df_shedule.fillna(value='Окно', inplace=True)
# display(df_shedule.isna().sum())
df_shedule.Profile.unique()



array(['Социология', 'Экономика', 'ГМУ', ' Сервис', 'Статистика',
       'Управление персоналом', 'Сервис'], dtype=object)

Проверка логики программы

In [172]:
def get_text_messages( message_group = input('Enter num group: '), message_day = input('Enter num day: ') ): #message_course = input('Enter num course: '),
    num_of_week_day = message_day.strip()
    # num_of_course = message_course.strip()
    num_of_group = message_group.strip()
    subject = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Subject'].values
    nm_lesson = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Num_Lesson'].values
    # print(f'{num_of_course} {num_of_group} {num_of_week_day}')
    
    if len(subject) > 0:
        i = 0
        for sub in subject:
            print(f"{df_shedule[df_shedule['Num_Day'] == int(num_of_week_day)]['Day'].values[0]} Пара # {nm_lesson[i]} по дисциплине: {sub}")
            i += 1

    else: 
        print(f' у Вас пар нет')


get_text_messages()

 у Вас пар нет


# Create bot Telegram

In [3]:
# 1)Create bot 2)Give token 3)Task like Neyro 4) Add pandas 5)Thin abot Logic 1010)task server Heroku -> RSU
load_dotenv()
TOKEN_BOT = os.getenv('TOKEN_BOT') 


In [ ]:
# next(iter(user_data.values()))
# first_key, first_value = next(iter(user_data.items()))
# print(f"Ключ: {first_key}, Значение: {first_value}")

Variant 3


Работает с датафреймом но кнопки не нравятся

In [82]:
univercity_df = pd.DataFrame(columns= ['course', "group", "nm"], data=[[1,9402, 25], [1,9403, 20], [2,9302,15]])
univercity_df.loc[len(univercity_df)] = [2,9308,18]
# univercity_df[univercity_df["course"] == 2]["group"].values[0:][0]

bot = telebot.TeleBot(TOKEN_BOT)

user_state = {}  # Словарь для хранения состояния каждого пользователя

@bot.message_handler(commands=['start']) # Функция для обработки команды /start
def start_command(message):
    user_id = message.from_user.id
    user_state[user_id] = {}  # Инициализация состояния для нового пользователя
    main_menu(message)

# Основная клавиатура с выбором курса
def main_menu(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn1 = types.KeyboardButton("Курс_1") # текст для передачи в Message след функции  message.text
    btn2 = types.KeyboardButton("Курс_2")
    keyboard.add(btn1, btn2)
    bot.send_message(message.chat.id, "Выберите курс:", reply_markup=keyboard)

# Подменю первой категории
def submenu_category1(message):
    nm_corse = message.text.split('_')[1]
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn11 = types.KeyboardButton(f'Группа_{univercity_df[univercity_df["course"] == int(nm_corse)]["group"].values[0:][0]}') # выводит кнопку с номером группы => передаем в след фнкц
    btn12 = types.KeyboardButton(f'Группа_{univercity_df[univercity_df["course"] == int(nm_corse)]["group"].values[0:][1]}')
    # list_btns = []
    keyboard.add(btn11, btn12)
    bot.send_message(message.chat.id, "Выберите группу:", reply_markup=keyboard)

# Подменю второй категории
def submenu_category2(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn21 = types.KeyboardButton("Опция 2.1")
    btn22 = types.KeyboardButton("Опция 2.2")
    keyboard.add(btn21, btn22)
    bot.send_message(message.chat.id, "Выберите опцию:", reply_markup=keyboard)

# Обработчик всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def process_message(message):
    user_id = message.from_user.id
    state = user_state.get(user_id, {})

    if not state:  # Если состояние пустое, значит, пользователь еще ничего не выбрал
        if message.text != "":
            state['course'] = message.text
            submenu_category1(message)
        # elif message.text == "Курс_2":
        #     state['coruse'] = "Курс_2"
        #     submenu_category1(message) # maybe 2 return
        else:
            bot.reply_to(message, "Пожалуйста, выберите курс.")
    else:
        # Состояние уже установлено, значит, пользователь выбирает опции внутри выбранной категории
        category = state['course']
        if message.text != "":
                state['group'] = message.text
                nm_grp = message.text.split('_')[1]
                bot.reply_to(message, f'''Ваш курс: {state['course']}, Ваша группа: {state['group']}! \n 
                            Численность группы {univercity_df[univercity_df['group'] == int(nm_grp)]['nm'].values[0:][0]}''')
        else:
                bot.reply_to(message, "Пожалуйста, выберите одну из опций.")

bot.polling()

В продакшн

In [76]:
bot = telebot.TeleBot(TOKEN_BOT)
user_state = {}  # Словарь для хранения состояния каждого пользователя

@bot.message_handler(commands=['start'])
def start_command(message):
    user_id = message.from_user.id
    user_state[user_id] = {}  # инициализация состояния
    user_state[user_id]['name'] = message.from_user.first_name
    name = user_state[user_id]['name']
    show_main_menu(message.chat.id, name)
    # print(bot.get_webhook_info())

def show_main_menu(chat_id, name):
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    btn1 = types.InlineKeyboardButton("Курс_1", callback_data="course_1")
    btn2 = types.InlineKeyboardButton("Курс_2", callback_data="course_2")
    btn3 = types.InlineKeyboardButton("Курс_3", callback_data="course_3")
    btn4 = types.InlineKeyboardButton("Курс_4", callback_data="course_4")

    keyboard.add(btn1, btn2, btn3, btn4)
    bot.send_message(chat_id, f'''Привет! {name} выбери курс ''', reply_markup=keyboard)

def show_submenu_group(chat_id, course_number):    # получаем список групп для курса
    groups = list(df_shedule[df_shedule["Courses"] == int(course_number)]["Group"].unique()) 
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    for grp in groups[:]: 
        btn = types.InlineKeyboardButton(f"Группа_{grp}", callback_data=f"group_{grp}")
        keyboard.add(btn)
    bot.send_message(chat_id, "Выберите группу:", reply_markup=keyboard)

def show_submenu_day(chat_id, group_number):
    name_days = pd.DataFrame({"1": ["Понедельник"], "2": ["Вторник"], "3": ["Среда"], "4": ["Четверг"], 
                                  "5": ["Пятница"], "6": ["Суббота"],  "7": ["Воскресенье"]}) #, index=range(7))
    name_days = name_days.stack().reset_index(drop=True)
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    i = 1
    for day in name_days: 
        btn = types.InlineKeyboardButton(f"{day}", callback_data=f"day_{i}")
        i += 1
        keyboard.add(btn)
    bot.send_message(chat_id, "Выберите день:", reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data.startswith(('course_', 'group_', 'day_')))
def callback_query(call):
    user_id = call.from_user.id
    data = call.data  # например, "course_1" или "group_101"
       
    if data.startswith("course_"):  # если выбираем курс
        course_number = data.split("_")[1]
        user_state[user_id]['course'] = f"Курс_{course_number}"  # отправляем подменю групп
        show_submenu_group(call.message.chat.id, course_number)
        bot.answer_callback_query(call.id)  # убираем "часики" на кнопке
   
    elif data.startswith("group_"):  # если выбираем группу
        group_number = data.split("_")[1]
        user_state[user_id]['group'] = f"Группа_{group_number}"
        user_state[user_id]['group_nm'] = group_number
        show_submenu_day(call.message.chat.id, group_number)

    elif data.startswith("day_"):  # если выбираем день
        day_number = data.split("_")[1]
        user_state[user_id]['day'] = f"День_{day_number}"  # отправляем подменю групп
        state = user_state[user_id]
        responses = df_shedule[(df_shedule['Group'] == int(state["group_nm"])) & (df_shedule['Num_Day'] == int(day_number))][["Num_Lesson" ,'Subject']].values
        for res in responses:
            response = f' Пара №{str(res[0:1])[1:-1]}: {str(res[1:2])[2:-2]}'
            bot.send_message(call.message.chat.id, response)
bot.polling()



In [6]:
df_shedule = pd.read_csv('SheduleFESU.csv', sep=";" )
df_shedule['Logo'].fillna(value="NoLogo", inplace=True)
df_shedule.shape[0] #['Logo'].isna().sum()


C:\Users\aigna\AppData\Local\Temp\ipykernel_21552\1035348208.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_shedule['Logo'].fillna(value="NoLogo", inplace=True)


400

Чтобы изменить режим работы вашего Telegram-бота с опроса (polling) на вебхуки (webhooks), нужно выполнить несколько шагов. Вот пошаговая инструкция:
1. Настройте веб-сервер: Вам потребуется сервер с доступом по HTTPS, чтобы Telegram мог отправлять обновления на ваш сервер. Часто используются такие сервисы, как Heroku, DigitalOcean или облачные функции.
2. Получите SSL-сертификат: Ваш сервер должен поддерживать HTTPS. Для этого можно использовать бесплатные решения, такие как Let's Encrypt.
3. Настройте вебхук: Используйте метод setWebhook Telegram API для установки вебхука. Это можно сделать с помощью HTTP-запроса. Примерный запрос выглядит так:
https://api.telegram.org/botВАШ_ТОКЕН_БОТА/setWebhook?url=ВАШ_URL_ВЕБХУКА

6911877132:AAGi1Ljjo3whs4dAQm12pqmDwUd1aJVCEHg
https://shedulebotrsu-a-ignatov.amvera.io
https://api.telegram.org/bot6911877132:AAGi1Ljjo3whs4dAQm12pqmDwUd1aJVCEHg/setWebhook?url=https://shedulebotrsu-a-ignatov.amvera.io

Замените ВАШ_ТОКЕН_БОТА на токен бота, а ВАШ_URL_ВЕБХУКА на URL вашего HTTPS сервера.

In [21]:
# import telebot
# 
bot = telebot.TeleBot("6911877132:AAGi1Ljjo3whs4dAQm12pqmDwUd1aJVCEHg")

# Удаление вебхука
bot.delete_webhook()

print("Вебхук отключен. Бот готов к работе в режиме опроса.")

Вебхук отключен. Бот готов к работе в режиме опроса.
